<a href="https://colab.research.google.com/github/sarathi-vs13/Deep-Learning/blob/main/Keras_Housing_price_prediction_case_study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Understanding Regression problems using Keras

In [26]:
from google.colab import files
uploaded = files.upload()


Saving boston.csv to boston (1).csv


In [27]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import warnings
import tensorflow as tf
warnings.filterwarnings('ignore')

In [ ]:
# with open (r"C:\Users\adarsha\Downloads\housing.csv") as fi:

#     file_contents = fi.readlines()
#     data_array = []
#     for li in file_contents:
#         data_array.append( li.split())

# data = pd.DataFrame(data_array,
#     columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD','TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']).

In [28]:
data = pd.read_csv('boston.csv')

In [ ]:
# - CRIM     per capita crime rate by town
# - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
# - INDUS    proportion of non-retail business acres per town
# - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
# - NOX      nitric oxides concentration (parts per 10 million)
# - RM       average number of rooms per dwelling
# - AGE      proportion of owner-occupied units built prior to 1940
# - DIS      weighted distances to five Boston employment centres
# - RAD      index of accessibility to radial highways
# - TAX      full-value property-tax rate per $10,000
# - PTRATIO  pupil-teacher ratio by town
# - B        1000(Bk - 0.63)^2 where Bk is the proportion of black people by town
# - LSTAT    % lower status of the population
# - MEDV     Median value of owner-occupied homes in $1000's

In [29]:
print(data.head())

      CRIM    ZN  INDUS  CHAS    NOX  ...    TAX  PTRATIO       B  LSTAT  MEDV
0  0.00632  18.0   2.31     0  0.538  ...  296.0     15.3  396.90   4.98  24.0
1  0.02731   0.0   7.07     0  0.469  ...  242.0     17.8  396.90   9.14  21.6
2  0.02729   0.0   7.07     0  0.469  ...  242.0     17.8  392.83   4.03  34.7
3  0.03237   0.0   2.18     0  0.458  ...  222.0     18.7  394.63   2.94  33.4
4  0.06905   0.0   2.18     0  0.458  ...  222.0     18.7  396.90   5.33  36.2

[5 rows x 14 columns]


##Train and Split

In [30]:
# Split data into test and train
train_dataset = data.sample(frac=0.7,random_state=0)
test_dataset = data.drop(train_dataset.index)

In [31]:
# Stastistical summary of data
# We will be using it to normalize the data
train_stats = train_dataset.describe()
train_stats.pop("MEDV") # because this is our target variable
train_stats = train_stats.transpose()
train_stats



,count,mean,std,min,25%,50%,75%,max
CRIM,354.0,3.767375,9.418497,0.00906,0.082757,0.274475,3.077295,88.9762
ZN,354.0,11.079096,23.070178,0.00000,0.000000,0.000000,12.500000,95.0000
INDUS,354.0,11.185254,6.646944,0.74000,5.860000,9.795000,18.100000,27.7400
CHAS,354.0,0.070621,0.256554,0.00000,0.000000,0.000000,0.000000,1.0000
NOX,354.0,0.554098,0.115748,0.38500,0.453000,0.538000,0.624000,0.8710
RM,354.0,6.265791,0.699380,3.56100,5.878250,6.175000,6.605500,8.7800
AGE,354.0,68.057627,27.953167,6.00000,45.100000,76.500000,93.750000,100.0000
DIS,354.0,3.844439,2.187514,1.12960,2.073700,3.207450,5.214600,12.1265
RAD,354.0,9.440678,8.569207,1.00000,4.000000,5.000000,20.000000,24.0000
TAX,354.0,407.500000,162.296676,187.00000,287.000000,337.000000,666.000000,711.0000


In [32]:
train_labels = train_dataset.pop('MEDV')   # Y_train
test_labels = test_dataset.pop('MEDV')

In [36]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

X_train_n = norm(train_dataset)
X_test_n = norm(test_dataset)

In [37]:
train_dataset.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
329,0.06724,0.0,3.24,0,0.460,6.333,17.2,5.2146,4,430.0,16.9,375.21,7.34
371,9.23230,0.0,18.10,0,0.631,6.216,100.0,1.1691,24,666.0,20.2,366.15,9.53
219,0.11425,0.0,13.89,1,0.550,6.373,92.4,3.3633,5,276.0,16.4,393.74,10.50
403,24.80170,0.0,18.10,0,0.693,5.349,96.0,1.7028,24,666.0,20.2,396.90,19.77
78,0.05646,0.0,12.83,0,0.437,6.232,53.7,5.0141,5,398.0,18.7,386.40,12.34


In [35]:
X_train_n.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
329,-0.392858,-0.480235,-1.195324,-0.275269,-0.812953,0.096098,-1.819387,0.626356,-0.634910,0.138635,-0.726275,0.234841,-0.751260
371,0.580233,-0.480235,1.040289,-0.275269,0.664389,-0.071193,1.142710,-1.223004,1.699028,1.592762,0.808798,0.140233,-0.439304
219,-0.387867,-0.480235,0.406916,3.622544,-0.035405,0.153292,0.870827,-0.219948,-0.518213,-0.810245,-0.958862,0.428337,-0.301132
403,2.233299,-0.480235,1.040289,-0.275269,1.200033,-1.310863,0.999614,-0.979029,1.699028,1.592762,0.808798,0.461334,1.019339
78,-0.394003,-0.480235,0.247444,-0.275269,-1.011659,-0.048316,-0.513631,0.534699,-0.518213,-0.058535,0.111037,0.351690,-0.039032


##Creating a model

In [40]:
# Build model
model = Sequential()
model.add(Dense(64, input_shape=(13,), activation='relu'))  # Hidden layer
model.add(Dense(64, activation='relu'))   # Hidden layer
model.add(Dense(1))  # Output layer

In [39]:
opt = tf.keras.optimizers.Adam()

In [41]:
# compile model
model.compile(loss='mean_squared_error', optimizer=opt,metrics=['mae','mse'])

In [42]:
# Fit
history = model.fit(X_train_n, train_labels, epochs = 1000,validation_split = 0.2)

Epoch 1/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - loss: 522.2401 - mae: 21.4430 - mse: 522.2401 - val_loss: 640.4071 - val_mae: 23.4015 - val_mse: 640.4071
Epoch 2/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 480.0193 - mae: 20.3506 - mse: 480.0193 - val_loss: 600.4253 - val_mae: 22.5321 - val_mse: 600.4253
Epoch 3/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 477.5442 - mae: 20.1415 - mse: 477.5442 - val_loss: 553.3268 - val_mae: 21.4795 - val_mse: 553.3268
Epoch 4/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 405.9706 - mae: 18.3099 - mse: 405.9706 - val_loss: 495.9277 - val_mae: 20.2097 - val_mse: 495.9277
Epoch 5/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 393.6051 - mae: 17.7614 - mse: 393.6051 - val_loss: 424.6224 - val_mae: 18.6153 - val_mse: 424.6224
Epoch 6/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 321.9485 - mae: 16.0051 - mse: 321.9485 - val_loss: 341.7093 - val_mae: 16.6402 - val_mse: 341.7093
Epoch 7/1000
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/